#Read The Data
here we read the data and orgainze it in a file to be ready

In [1]:
__author__ = 'aqeel'
'''Train and evaluate a simple MLP on the Souq.com Reviews newswire topic classification task.
GPU run command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python examples/NNClassifiyReviews.py
CPU run command:
    python examples/NNClassifiyReviews.py
'''
import numpy as np
np.random.seed(1337)
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Merge, Dropout, RepeatVector,MaxoutDense,Activation
from keras.layers import recurrent
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import math
import random
random.seed(1337)

Using Theano backend.


#Read The Data

In [2]:
mydata = pd.read_csv('astronomy.stackexchange.com/All.csv',sep=',',quotechar='"')

In [3]:
mydata['body'].head(1)[0]
for i in range(len(mydata)):
    if isinstance(mydata['body'][i], float):
        print i,mydata['body'][i]
    
    

    

In [4]:
MORE_THAN_MEAN_BY= 100
mydata['a_words'] = map(lambda x:x.split(),mydata['body'])
mydata['q_words'] = map(lambda x:x.split(),mydata['q_body'])
mydata['q_bdlen'] = map(lambda x:len(x),mydata['q_words'])
mydata['a_bdlen'] = map(lambda x:len(x),mydata['a_words'])
Q_MAX = max(mydata['q_bdlen'])
A_MAX = max(mydata['a_bdlen'])
Q_MEAN = int(math.floor(np.mean(mydata['q_bdlen']))+MORE_THAN_MEAN_BY)
A_MEAN=int(math.floor(np.mean(mydata['a_bdlen']))+MORE_THAN_MEAN_BY)
print min(mydata['q_bdlen']),Q_MAX,Q_MEAN-MORE_THAN_MEAN_BY
print min(mydata['a_bdlen']),A_MAX,A_MEAN-MORE_THAN_MEAN_BY
np.median(mydata['q_score'])


3 3095 50
3 2154 117


3.0

In [5]:
len(mydata)

4536

In [6]:
mydata.head(1)

,id,score,parent,body,q_score,q_ansrcnt,q_ViewCount,q_body,a_words,q_words,q_bdlen,a_bdlen
0,8200,8,8192,l4 l5 point pair bodi onli stabl larger bodi l...,6,1,277,usual exampl lagrang point mechan one common e...,"[l4, l5, point, pair, bodi, onli, stabl, large...","[usual, exampl, lagrang, point, mechan, one, c...",66,35


#Split the data (Train,Test)

def GetData(splitper=0.2):
    global mydata
    splitper = int(math.floor(splitper * len(mydata)) + 1)
    #Shuffle the data
    mydata = mydata.iloc[np.random.permutation(len(mydata))]
    return mydata['q_body'][splitper:].tolist(),mydata['body'][splitper:].tolist(),\
    mydata['score'][splitper:].tolist(),mydata['q_body'][:splitper].tolist(),\
    mydata['body'][:splitper].tolist(),mydata['score'][:splitper].tolist()

In [22]:
def GetDataW(splitper=0.2):
    global mydata
    splitper = int(math.floor(splitper * len(mydata)) + 1)
    #Shuffle the data
    mydata = mydata.iloc[np.random.permutation(len(mydata))]
    mydata[:splitper].to_csv('test.csv')
    return mydata['q_words'][splitper:].tolist(),mydata['a_words'][splitper:].tolist(),\
    mydata['score'][splitper:].tolist(),mydata['q_words'][:splitper].tolist(),\
    mydata['a_words'][:splitper].tolist(),mydata['score'][:splitper].tolist()

In [23]:
print ('Loading Data....')
Qx_trn,Ax_trn,y_trn,Qx_test,Ax_test,y_test= GetDataW(0.2)
print ('Data Loaded')

Loading Data....
Data Loaded


In [9]:
All_Vocabulary={}
for i in Qx_trn+Ax_trn+Qx_test+Ax_test:
    for element in i:
        All_Vocabulary[element]=True

All_Vocabulary= All_Vocabulary.keys()
print len(All_Vocabulary)

60644


AllQuestionsWords = mydata.drop_duplicates('parent')['q_words']

print('Vectorizing sequence data...')
tokenizer = Tokenizer(nb_words=max_words)
tokenizer.fit_on_texts(AllQuestionsWords+mydata['body'].tolist())

In [10]:
vocab_size = len(All_Vocabulary)+1
word_idx = dict((c, i + 1) for i, c in enumerate(All_Vocabulary))


print('Vectorizing sequence data...')
tokenizer = Tokenizer(nb_words=10)
tokenizer.fit_on_texts(mydata['body'][0])
t=tokenizer.texts_to_sequences(mydata['body'][0])
'''t[np.all(t==[9], axis=0)]'''
print [x[0] for x in t if x]

In [11]:
def Vectorize():
    global Qx_trn,Ax_trn,Qx_test,Ax_test,y_trn,y_test
    
    for i in range (0,len(Qx_trn)):
        Qx_trn[i] = [word_idx[l] for l in Qx_trn[i]]
    Qx_trn= pad_sequences(Qx_trn,Q_MEAN)
    
    for i in range (0,len(Ax_trn)):
        Ax_trn[i] = [word_idx[l] for l in Ax_trn[i]]
    Ax_trn =pad_sequences(Ax_trn,A_MEAN)
    
    for i in range (0,len(Qx_test)):
        Qx_test[i] = [word_idx[l] for l in Qx_test[i]]
    Qx_test= pad_sequences(Qx_test,Q_MEAN)
    
    for i in range (0,len(Ax_test)):
        Ax_test[i] = [word_idx[l] for l in Ax_test[i]]
    Ax_test = pad_sequences(Ax_test,A_MEAN)
    y_trn = np.array(y_trn)
    y_test = np.array(y_test)

In [12]:
Vectorize()

In [13]:
print('All Vocabulary  = {}'.format(vocab_size))
print('Train Questions.shape = {},Test Questions.shape{}'.format(Qx_trn.shape,Qx_test.shape))
print('Train Answers.shape = {},Test Answers.shape{}'.format(Ax_trn.shape,Ax_test.shape))
print('Train_Y.shape = {},Test_Y.shape{}'.format(y_trn.shape,y_test.shape))
print('Questions Max Length, Answers Max Length = {}, {}'.format(Q_MAX, A_MAX))

All Vocabulary  = 60645
Train Questions.shape = (3628, 150),Test Questions.shape(908, 150)
Train Answers.shape = (3628, 217),Test Answers.shape(908, 217)
Train_Y.shape = (3628,),Test_Y.shape(908,)
Questions Max Length, Answers Max Length = 3095, 2154


#Sequence The Training and Testing Set
Qx_trn = tokenizer.texts_to_sequences(Qx_trn)
Ax_trn= tokenizer.texts_to_sequences(Ax_trn)
Qx_test = tokenizer.texts_to_sequences(Qx_test)
Ax_test= tokenizer.texts_to_sequences(Ax_test)
print ('Sequences : Q_train:{},A_train:{},Q_test:{},A_test:{}'\
       .format(len(Qx_trn),len(Ax_trn),len(Qx_test),len(Ax_test)))

Qx_trn = tokenizer.sequences_to_matrix(Qx_trn, mode=sequencemode)
Ax_trn= tokenizer.sequences_to_matrix(Ax_trn, mode=sequencemode)
Qx_test = tokenizer.sequences_to_matrix(Qx_test, mode=sequencemode)
Ax_test= tokenizer.sequences_to_matrix(Ax_test, mode=sequencemode)
       
print ('Shapes : Q_train:{},A_train:{},Q_test:{},A_test:{}'\
       .format(Qx_trn.shape,Ax_trn.shape,Qx_test.shape,Ax_test.shape))

In [14]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
#1-inf
BATCH_SIZE = 32
#1-inf
EPOCHS = 2
#Done
#SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax
theoptimizer = 'RMSprop'
#DONE
#0.1-0.9
thedropout =0.5
#DONE
#softmax,softplus,relu,tanh,sigmoid,hard_sigmoid,linear,
FirstActivation = 'relu'
SecondActivation='softmax'
#DONE
#mean_squared_error / mse,root_mean_squared_error / rmse,mean_absolute_error / mae,mean_absolute_percentage_error / mape
#mean_squared_logarithmic_error / msle,squared_hinge, hinge,binary_crossentropy: Also known as logloss,categorical_crossentropy: Also known as multiclass logloss. Note: using this objective requires that your labels are binary arrays of shape (nb_samples, nb_classes).
#poisson: mean of (predictions - targets * log(predictions))# cosine_proximity: the opposite (negative) of the mean cosine proximity between predictions and targets.
theloss='mse'
#DONE
#modes: binary, count, tfidf, freq (best test accuracy count),(best Train Accuracy binary)
sequencemode = 'count'
#======================
results = open('results.txt','a')
results.write('\n'+str(BATCH_SIZE)+','+str(EPOCHS)+','+theoptimizer+','+str(thedropout)+','+FirstActivation+','+SecondActivation + ','+theloss)

In [15]:
print('Build model...')

Questions = Sequential()
Questions.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE, input_length=Q_MEAN, mask_zero=True))
Questions.add(Dropout(thedropout))
Questions.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))

Answers = Sequential()
Answers.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE, input_length=A_MEAN,mask_zero=True))
Answers.add(Dropout(thedropout))
Answers.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))

Build model...


In [17]:
Questions.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
embedding_1 (Embedding)            (None, 150, 50)     3032250     embedding_input_1[0][0]          
____________________________________________________________________________________________________
dropout_1 (Dropout)                (None, 150, 50)     0           embedding_1[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                      (None, 50)          20200       dropout_1[0][0]                  
Total params: 3052450
____________________________________________________________________________________________________


In [18]:
print Questions.output_shape
print Answers.output_shape
m = Merge([Questions, Answers], mode='concat')
print m.output_shape
results.write('\n Test loss / test accuracy = {:.4f} / {:.4f}'.format(1.0, 1.0))
results.close()

(None, 50)
(None, 50)
(None, 100)


In [19]:
model = Sequential()
model.add(Merge([Questions, Answers], mode='concat'))
#model.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer=theoptimizer,
              loss=theloss,
              metrics=['accuracy'])

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
embedding_1 (Embedding)            (None, 150, 50)     3032250                                      
____________________________________________________________________________________________________
dropout_1 (Dropout)                (None, 150, 50)     0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                      (None, 50)          20200                                        
____________________________________________________________________________________________________
embedding_2 (Embedding)            (None, 217, 50)     3032250                                      
___________________________________________________________________________________________

In [16]:


print('Training')
hist = model.fit([Qx_trn, Ax_trn], y_trn, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, validation_split=0.05)
loss, acc = model.evaluate([Qx_test, Ax_test], y_test, batch_size=BATCH_SIZE)
results.write('\n Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training


KeyboardInterrupt: 

#This results copied after executing the code on juur.grid.eenet.ee
BatchSize:32,EPOCH:2,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:150,AVectorLength:217
 Train loss / Train accuracy = 15.5472828403 / 0.158765159876
 Test loss / test accuracy = 20.7112115532 / 0.140969163061
BatchSize:32,EPOCH:2,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:250,AVectorLength:317
 Train loss / Train accuracy = 16.2962467799 / 0.134785005521
 Test loss / test accuracy = 21.181909788 / 0.135462555132
BatchSize:32,EPOCH:2,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:50,AVectorLength:117
 Train loss / Train accuracy = 16.3322546338 / 0.136714443228
 Test loss / test accuracy = 21.1259150232 / 0.136563876718
BatchSize:32,EPOCH:200,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:50,AVectorLength:117
 Train loss / Train accuracy = 0.586722035606 / 0.909316427784
 Test loss / test accuracy = 21.4848160261 / 0.147577092544
BatchSize:32,EPOCH:100,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:50,AVectorLength:117
 Train loss / Train accuracy = 0.66182425444 / 0.881477398015
 Test loss / test accuracy = 21.3947613082 / 0.157488986915
BatchSize:32,EPOCH:50,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:50,AVectorLength:117
 Train loss / Train accuracy = 0.797981166787 / 0.780319735391
 Test loss / test accuracy = 20.6134651453 / 0.150881057269
BatchSize:32,EPOCH:50,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:50,AVectorLength:117
 Train loss / Train accuracy = 0.835777364992 / 0.766538037494
 Test loss / test accuracy = 20.5429394088 / 0.138766519824
BatchSize:32,EPOCH:150,Optimizer:Adam,Dropout:0.5,2ndActivation:softmax,thelossmse,QVectorLength:50,AVectorLength:117
 Train loss / Train accuracy = 0.693766769839 / 0.877618522618
 Test loss / test accuracy = 21.203188287 / 0.132158590341